# **12.3 Implement an ISBN Cache**
---
- ISBN: International Standard Book Number
    - unique commercial book identifier 
    - string length = `10`
        - first `9` characters are digits 
        - last character = check character 
            - `sum` of first `9` digits
            - `mod 11` with `10` represented by `X`
    - *modern ISBNs are now `13` digits and the check digit taken `mod 10`*
- Create a cache for looking up prices of books via their ISBN
- Assumptions:
    - ISBNs = `10` digits
    - ISBNs and prices all POSITIVE INTEGERS
- implement `lookup`, `insert`, and `erase` 
    - `insert`: if ISBN already present -> do not update price, update entry to most recently used ISBN
    - `lookup`: given ISBN return the corresponding price 
        - if element is not present: return -1
        - if ISBN present: update entry to most recently used ISBN
    - `erase`: remove specified ISBN and corresponding value from the case 
        - return `True` if the ISBN was present
        - return `False` if it was not 
- use LRU (least recently used) policy for cache eviction 

---
### Just a Hash Table
- for each ISBN (key), we store a value (price), and a timestamp (count corresponding to when the ISBN was most recently inserted/looked up)
- Time Complexity:
    - Lookup/Delete: `O(1)`
    - Inserts: `O(1)` until the cache is full 
        - once cache is full
            - find the LRU ISBN
            - evict the LRU ISBN to make place for the new entry 
            - `O(n)` time where `n` is the cache size 
            - have to scan all entries to find the one with the smallest time stamp 
            
---
## Record ISBNs in a queue *and* a Hash Table
- avoid processing all ISBNs
- conceptually ISBNs ordered by when they were most recently used 
    - using a queue garuntees this organization
- We only need to be able to find the oldest ISBN efficiently 
- for each ISBN:
    - store a reference to location in the queue 
    - at each ISBN `lookup` -> move to the front of the queue
        - requires a **linked list** implementation of the queue
        - items in the middle of the queue moved to the head 
- insert an ISBN that is already present:
    - if the insert results in the queue size exceeding `n`, ISBN at the tail of the queue is deleted from the cache

In [1]:
import collections

In [2]:
class LRU_Cache:
    
    def __init__(self, capacity: int) -> None:
        self._isbn_price_table: collections.OrderedDict[int, int] = collections.OrderedDict()
        self._capacity = capacity
    
    def lookup(self, isbn: int) -> int: 
        if isbn not in self._isbn_price_table:
            return -1 
        # pops value of ISBN (key) which is the price of the book 
        price = self._isbn_price_table.pop(isbn)
        # updates price of the book
        self._isbn_price_table = price 
        return price 
    
    def insert(self, isbn: int, price: int) -> None:
        if isbn in self._isbn_price_table:
            # get the price value 
            price = self._isbn_price_table.pop(isbn)
            
        elif len(self._isbn_price_table) == self._capacity:
            # .popitem() -> remove and return a (key,value) pair from the dictionary 
            # pairs returned in LIFO (queue) order 
            self._isbn_price_table.popitem(last=False)
            
        # replace the price value 
        self._isbn_price_table[isbn] = price 
    
    def erase(self, isbn: int) -> bool:
        return self._isbn_price_table.pop(isbn, None) is not None           
            

In [3]:
lru = LRU_Cache(4)
lru.insert(1112223334,11)
lru.insert(2223334445,12)
lru.insert(3334445556,13)
lru.erase(1112223334)

True

In [4]:
lru.insert(4446667778,14)
lru.lookup(1112223334)

-1

#### Time Complexity: `O(1)`
- `O(1)` for each lookup, hash table lookup, and updating the queue -> evens out to `O(1)`